In [33]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from datasets import ClassLabel
import torch

In [2]:
model_synth = BertForSequenceClassification.from_pretrained(f"../models/saved_model_minimal_synth")
tokenizer_synth = BertTokenizer.from_pretrained(f"../models/saved_tokenizer_synth")

In [22]:
model_baseline = BertForSequenceClassification.from_pretrained(f"../models/saved_model_minimal")
tokenizer_baseline = BertTokenizer.from_pretrained(f"../models/saved_tokenizer")

In [ ]:
# model_twitch_labeled = BertForSequenceClassification.from_pretrained(f"../models/saved_model_minimal_twitch_labeled")
# tokenizer_twitch_labeled = BertTokenizer.from_pretrained(f"../models/saved_tokenizer_twitch_labeled")

In [29]:
test_set = pd.read_csv("../Data/combined_twitch_data_labeled_by_GPT.csv")

In [30]:
def evaluate_model(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    pred = outputs.logits.argmax(dim=1).item()
    return "Offensive" if pred == 1 else "Not Offensive"

In [34]:
inputs_synth = tokenizer_synth(
    list(test_set['message']),
    padding=True,
    truncation=True,
    return_tensors='pt'
)
inputs_baseline = tokenizer_baseline(
    list(test_set['message']),
    padding=True,
    truncation=True,
    return_tensors='pt'
)

# Move to same device as model
device = model_synth.device
inputs_synth = {k: v.to(device) for k, v in inputs_synth.items()}
inputs_baseline = {k: v.to(device) for k, v in inputs_baseline.items()}

# Inference without gradient
with torch.no_grad():
    preds_synth = model_synth(**inputs_synth).logits.argmax(dim=1).cpu().numpy()
    preds_baseline = model_baseline(**inputs_baseline).logits.argmax(dim=1).cpu().numpy()

test_set['pred_synth'] = preds_synth
test_set['pred_baseline'] = preds_baseline

RuntimeError: [enforce fail at alloc_cpu.cpp:121] data. DefaultCPUAllocator: not enough memory: you tried to allocate 3108725760 bytes.

In [ ]:
# text = "why are you so good at this"
# inputs = tokenizer_synth(text, return_tensors="pt")
# outputs = model_synth(**inputs)
# pred = outputs.logits.argmax(dim=1).item()
# print("Offensive" if pred == 1 else "Not Offensive")

Offensive


In [ ]:
# text = "hello"
# inputs = tokenizer_baseline(text, return_tensors="pt")
# outputs = model_baseline(**inputs)
# pred = outputs.logits.argmax(dim=1).item()
# print("Offensive" if pred == 1 else "Not Offensive")

Not Offensive
